In [ ]:
import numpy as np
from numba import jit, njit
import matplotlib.pyplot as plt
from random import randint
from sklearn.datasets import make_blobs
from utility import *
from tqdm import tqdm
from sklearn.cluster import k_means

In [ ]:
@njit
def neighbors_clustering(sol, points, K):
    neighbors = []

    q = int(len(sol)/100)

    choices = np.arange(len(sol))
    orig_val = squared_inner_distance(sol, points, K)
    centroids = calc_centroids(sol, points, K)

    for i in range(q):
        choice = np.random.randint(len(choices))
        choices = np.delete(choices, choice)
        for k in range(K):
            new_sol = sol.copy()
            if(k != new_sol[choice]):
                new_sol[choice] = k
                new_val = orig_val - np.linalg.norm(points[choice]-centroids[sol[choice]]) + np.linalg.norm(points[choice]-centroids[k])
                neighbors.append((new_sol,new_val))


    choices1 = np.arange(len(sol))
    choices2 = np.arange(len(sol))
    for i in range(q**2):
        choice1 = np.random.randint(len(choices1))
        choices1 = np.delete(choices1, choice1)
        choice2 = np.random.randint(len(choices2))
        choices2 = np.delete(choices2, choice2)

        new_sol = sol.copy()
        new_sol[choice1], new_sol[choice2] = new_sol[choice2], new_sol[choice1]

        new_val = orig_val - np.linalg.norm(points[choice1]-centroids[sol[choice1]]) - np.linalg.norm(points[choice2]-centroids[sol[choice2]]) + np.linalg.norm(points[choice1]-centroids[sol[choice2]]) + np.linalg.norm(points[choice2]-centroids[sol[choice1]])
        neighbors.append((new_sol,new_val))
        
    return neighbors


@njit
def local_search(base_sol, points, K, verbose = True):
    old_sol = base_sol
    base_val = squared_inner_distance(old_sol, points, K)
    iter = 1
    same_sol = 0

    while True:
        neighbourhood = neighbors_clustering(old_sol, points, K)
        best_val = squared_inner_distance(old_sol, points , K)
        best_sol = old_sol

        if verbose:
            print("Iteration number:", iter, "Valore percentuale:", best_val/base_val*100, "%")

        for sol,val in neighbourhood:
            if(val < best_val):
                best_val = val
                best_sol = sol
        
        if(best_sol is old_sol):
            same_sol = same_sol + 1
            if(same_sol == 100):
                break
        else:
            same_sol = 0
            old_sol = best_sol

        iter = iter+1
    return old_sol

In [ ]:
K = 10
N = 1000

points, centroids = make_blobs(n_samples=N, centers=K, n_features=2, random_state=np.random.randint(10))

sol = np.random.randint(K, size = N)
sol = local_search(sol, points, K)

print("Il valore di f.obj ottenuta è: {:.5E}".format(squared_inner_distance(sol, points, K)))

printR2sol(points, sol, K)

In [ ]:
n_points = [500,1000,1000,1000,1000,1000,1500,1500,2000,3000,5000,10000]
n_clusters = [5,2,4,5,6,7,5,10,5,5,5,10]
dim_points = [32,32,32,32,20,20,18,18,18,16,16,16]


vals = []

for test in tqdm(range(1,13)):
    points = load_points(f'C:/Users/franc/Documents/GitHub/Ricerca_Operativa_2022/Ricerca_Operativa_2022/benchmark/benchmark{test}.txt')
    N = len(points)
    K = n_clusters[test-1]
    sol = np.random.randint(K, size = N) #soluzione iniziale
    sol = local_search(sol, points, K, False)
    val = squared_inner_distance(sol, points, K)
    vals.append(val)
    print(val)
print(vals)
with open("risultatiLS.txt", 'w') as file:
    file.write("Local search:\n")
    file.write(str(vals))